在进行债券市场监测时，可以结合以下一些更深入的指标，帮助全面评估市场交易情况和潜在风险：

### 1. **流动性指标**
   - **日均交易量波动**：观察每个债券的日均交易量变化，波动较大的债券可能流动性不足。
   - **买卖价差 (Bid-Ask Spread)**：计算每笔交易的买入和卖出价差，价差大的债券通常流动性较低。
   - **活跃度 (Turnover Rate)**：通过每只债券的交易量与其总发行量的比率，判断其活跃度。

### 2. **价格波动性**
   - **日内价格波动**：计算每日的最高价与最低价的波动率，监测债券是否有异常的价格波动。
   - **标准差 (Price Volatility)**：计算各债券的价格标准差，以评估债券的价格稳定性。
   - **偏离均价幅度 (Price Deviation)**：计算交易价格偏离近期（如1天或5天）均价的幅度，识别异常波动。

### 3. **收益率相关指标**
   - **收益率偏离度**：比较每个债券的到期收益率和市场平均收益率，识别明显偏离市场的债券。
   - **收益率期限结构 (Yield Curve Position)**：债券在收益率曲线中的位置，观察其是否符合常规期限结构，以便识别潜在的错配。
   - **隐含收益率波动性**：跟踪收益率的历史波动性，判断债券的收益率风险。

### 4. **交易行为分析**
   - **单日大额交易比例**：统计单日内单笔交易量占比超出某一阈值（如5%）的情况，识别大额交易。
   - **高频交易特征**：观察高频交易模式，尤其是短时间内频繁买卖的情况，可能是流动性不足或投机行为。
   - **机构集中度**：按买卖双方机构统计交易分布，判断是否存在某些机构主导市场的情况。

### 5. **市场异常事件检测**
   - **异常交易量**：通过历史交易量数据识别异常高或低的交易量，可能表明市场操纵或信息泄露。
   - **价格跳跃检测**：在较短时间段内检测价格是否出现跳跃式变化，可能代表突发事件或市场情绪波动。
   - **连锁交易 (Chain Trading)**：识别某个债券的连续交易链，可能暗示潜在的市场操控行为。

### 6. **债券风险评估**
   - **信用利差变动 (Credit Spread)**：跟踪债券的信用利差波动情况，信用利差突然变化可能暗示信用风险。
   - **违约概率**：基于市场信息或外部评级数据，计算各债券的违约概率。
   - **赎回风险 (Call/Put Risk)**：分析具有赎回条款债券的提前赎回或提前出售风险。

### 7. **其他市场环境指标**
   - **市场情绪指标 (Sentiment Index)**：利用情绪分析方法，对新闻或社交媒体中关于债券的讨论进行情绪分析。
   - **行业对比**：将不同行业的债券交易指标进行横向对比，识别行业间的差异性和趋势。
   - **宏观经济影响因素**：将利率、通胀、货币政策变化等因素作为外部变量，观察其对债券市场的影响。

这些指标可以帮助更全面地监测债券市场的交易活动、流动性情况、价格波动及潜在风险，从而实现对市场异常和潜在风险的实时监控。

以下是实现上述指标的Python代码示例。假设你已经有了包含债券交易数据的DataFrame `df`，其中包含如下列：`bond_cd`（债券代码），`net_prc`（净价），`yld_to_mrty`（到期收益率），`nmnl_vol`（交易量），`trade_date`（交易日期），以及其他需要的列。

### 1. 导入所需的库

```python
import pandas as pd
import numpy as np
from scipy.stats import zscore
```

### 2. 流动性指标

#### 日均交易量波动

```python
# 计算每只债券的日均交易量波动
df['trade_date'] = pd.to_datetime(df['trade_date'])
daily_volume = df.groupby(['bond_cd', 'trade_date'])['nmnl_vol'].sum()
daily_volatility = daily_volume.groupby('bond_cd').std()
```

#### 买卖价差

假设数据中有买入价格 `bid_price` 和卖出价格 `ask_price`：

```python
df['bid_ask_spread'] = df['ask_price'] - df['bid_price']
spread_avg = df.groupby('bond_cd')['bid_ask_spread'].mean()
```

### 3. 价格波动性

#### 日内价格波动

```python
daily_price_range = df.groupby(['bond_cd', 'trade_date'])['net_prc'].agg(lambda x: x.max() - x.min())
```

#### 偏离均价幅度

```python
df['price_deviation'] = abs(df['net_prc'] - df.groupby('bond_cd')['net_prc'].transform('mean'))
```

### 4. 收益率相关指标

#### 收益率偏离度

```python
market_avg_yield = df['yld_to_mrty'].mean()
df['yield_deviation'] = abs(df['yld_to_mrty'] - market_avg_yield)
```

### 5. 交易行为分析

#### 单日大额交易比例

设定一个阈值（例如总交易量的5%）：

```python
threshold = 0.05 * df['nmnl_vol'].sum()
large_trades = df[df['nmnl_vol'] > threshold].groupby('bond_cd').size()
```

#### 高频交易特征

```python
high_freq_trades = df.groupby(['bond_cd', 'trade_date']).size()
high_freq_trades = high_freq_trades[high_freq_trades > 10]  # 例如一天内超过10次交易
```

### 6. 市场异常事件检测

#### 异常交易量

使用 `zscore` 检测异常：

```python
df['volume_zscore'] = df.groupby('bond_cd')['nmnl_vol'].transform(lambda x: zscore(x, nan_policy='omit'))
outliers = df[df['volume_zscore'] > 3]  # Z-score 大于3视为异常
```

#### 价格跳跃检测

```python
df['price_jump'] = df.groupby('bond_cd')['net_prc'].diff().abs()
price_jumps = df[df['price_jump'] > df['price_jump'].mean() + 3 * df['price_jump'].std()]
```

### 7. 债券风险评估

#### 信用利差变动

假设数据中有债券的信用利差 `credit_spread`：

```python
df['spread_change'] = df.groupby('bond_cd')['credit_spread'].diff()
spread_outliers = df[df['spread_change'].abs() > 3 * df['spread_change'].std()]
```

#### 违约概率

根据外部模型或评级计算违约概率（假设生成的概率为 `default_prob`）：

```python
df['default_prob'] = ...  # 引入违约概率模型或数据
high_risk_bonds = df[df['default_prob'] > 0.1]  # 假设高风险阈值为10%
```

### 8. 其他市场环境指标

#### 市场情绪指标 (例如通过外部情绪分析数据)

```python
df['sentiment_score'] = ...  # 引入外部市场情绪数据
sentiment_anomalies = df[df['sentiment_score'] < -0.5]  # 假设负面情绪分数低于-0.5
```

这些代码可以作为初步监测工具的实现。如果需要更复杂的分析和模型，可以基于此基础扩展数据处理、引入统计模型或机器学习技术，以实现更全面的监控系统。

In [20]:
import pandas as pd
import numpy as np
from scipy.stats import zscore

# 读取数据
df = pd.read_csv('ndm_transaction_list_all_20230705.csv')

# 数据清洗与转换
# 确保交易量和价格相关的字段为数值
df['nmnl_vol'] = pd.to_numeric(df['nmnl_vol'], errors='coerce')
df['net_prc'] = pd.to_numeric(df['net_prc'], errors='coerce')
df['trade_date'] = pd.to_datetime(df['dl_tm'])
# 按债券代码分组
bond_stats = df.groupby('bond_cd').agg(
    trade_count=('bond_cd', 'size'),  # 交易笔数
    total_volume=('nmnl_vol', 'sum'),  # 总交易量
    avg_price=('net_prc', 'mean'),     # 平均交易价格
)

# 计算偏移均价的交易笔数占比
# 定义偏移均价的标准 (假设为价格偏离均价超过一定阈值，比如1%）
price_threshold = 0.01

# 计算每条交易偏离均价的情况
df['is_offset'] = (abs(df['net_prc'] - df.groupby('bond_cd')['net_prc'].transform('mean')) / df.groupby('bond_cd')['net_prc'].transform('mean')) > price_threshold

# 计算偏移均价交易笔数占比
offset_stats = df.groupby('bond_cd')['is_offset'].mean() * 100  # 转为百分比

# 合并结果
bond_stats = bond_stats.join(offset_stats.rename('offset_trade_ratio'))

# 输出结果
print(bond_stats.head())

# 生成监测指标表
monitoring_df = pd.DataFrame()

# 债券代码
monitoring_df['bond_cd'] = df['bond_cd'].unique()

# 计算每只债券的日均交易量波动
daily_volume = df.groupby(['bond_cd', 'trade_date'])['nmnl_vol'].sum()
monitoring_df['daily_volume_volatility'] = daily_volume.groupby('bond_cd').std().values

# 买卖价差
# df['bid_ask_spread'] = df['ask_price'] - df['bid_price']
# spread_avg = df.groupby('bond_cd')['bid_ask_spread'].mean()
# monitoring_df['average_bid_ask_spread'] = spread_avg.values

# 价格波动性
daily_price_range = df.groupby(['bond_cd', 'trade_date'])['net_prc'].agg(lambda x: x.max() - x.min())
monitoring_df['daily_price_range'] = daily_price_range.groupby('bond_cd').mean().values

df['price_deviation'] = abs(df['net_prc'] - df.groupby('bond_cd')['net_prc'].transform('mean'))
monitoring_df['average_price_deviation'] = df.groupby('bond_cd')['price_deviation'].mean().values

# 收益率偏离度
market_avg_yield = df['yld_to_mrty'].mean()
df['yield_deviation'] = abs(df['yld_to_mrty'] - market_avg_yield)
monitoring_df['average_yield_deviation'] = df.groupby('bond_cd')['yield_deviation'].mean().values

# 单日大额交易比例
# threshold = 0.05 * df['nmnl_vol'].sum()
# large_trades = df[df['nmnl_vol'] > threshold].groupby('bond_cd').size()
# monitoring_df['large_trade_ratio'] = large_trades / df.groupby('bond_cd').size().values

# 异常交易量
df['volume_zscore'] = df.groupby('bond_cd')['nmnl_vol'].transform(lambda x: zscore(x, nan_policy='omit'))
monitoring_df['abnormal_volume'] = df['volume_zscore'] > 3  # Z-score > 3 as outliers

# 价格跳跃检测
# df['price_jump'] = df.groupby('bond_cd')['net_prc'].diff().abs()
# price_jumps = df[df['price_jump'] > df['price_jump'].mean() + 3 * df['price_jump'].std()]
# monitoring_df['price_jump_count'] = price_jumps.groupby('bond_cd').size().reindex(monitoring_df['bond_cd'], fill_value=0).values



         trade_count  total_volume   avg_price  offset_trade_ratio
bond_cd                                                           
101908             1    30000000.0  100.700000                 0.0
101962             1   100000000.0  101.017300                 0.0
101963             1    30000000.0  101.017300                 0.0
104505             3   180000000.0  104.101767                 0.0
104675             1   100000000.0  101.344800                 0.0


/Users/dennis/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3505: DtypeWarning: Columns (69,83,110,121,122,123,124,125,128) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


AttributeError: 'SeriesGroupBy' object has no attribute 'values'

In [21]:
monitoring_df

,bond_cd,daily_volume_volatility
0,2305528,NaN
1,102381121,NaN
2,2305622,NaN
3,2228009,0.000000e+00
4,2271766,NaN
...,...,...
1847,1880295,NaN
1848,112210336,1.902379e+07
1849,12380301,NaN
1850,102281365,1.131371e+07
